# preprocess LEMON raw signal

In [1]:
import shutil
import sys
import glob
import tarfile
import os
import re
import mne
import pandas as pd
import numpy as np
from tqdm import tqdm
from directory_tree import display_tree
from mne_connectivity import spectral_connectivity_epochs
from mne.datasets import sample
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

from mne_connectivity import spectral_connectivity_epochs
from mne.datasets import sample

Set file path

In [14]:
save_dir = '/workspace/brail_poc3/experiments/Lemon_Raw' # raw tar.gz zip file directroy, the same path with DownloadRawScript

# New path create
output_folder = '/workspace/brail_poc3/experiments/Lemon_PreRaw' # specificate unzip file folder
# Preprocessed LEMON eeg for location info (Download from github)
locFile_dir = '/workspace/brail_poc3/experiments/Lemon_PreEEGLAB/sub-032301/sub-032301/sub-032301_EC.set' 
# specificate after preprocessing folder, output preprocessing EEG files in .fif
preprocessed_dir = '/workspace/brail_poc3/experiments/Lemon_PreDict' 

Step1: Unzip file

1 file error: sub-032337 (download error?)

In [ ]:
# File paths
# tar_filename = '/workspace/brail_poc3/experiments/Lemon_Raw/sub-032301 (1).tar.gz'
tar_filename_list = glob.glob(save_dir+'/*.tar.gz')

error_files = []

# Extract each tar.gz file in the list
for tar_filename in tqdm(tar_filename_list):
    try:
        with tarfile.open(tar_filename, 'r:gz') as tar:
            tar.extractall(path=output_folder)

        # Print the list of extracted files
        extracted_files = os.listdir(output_folder)
        print(f"Extracted contents from {tar_filename} to {output_folder}:")

#         for file in extracted_files:
#             print(os.path.join(output_folder, file))

    except EOFError as e:
        print(f"Error extracting {tar_filename}: {e}")
        error_files.append(tar_filename)

# Print the list of files that caused errors
if error_files:
    print("Files that caused errors:")
    for error_file in error_files:
        print(error_file)

In [15]:
# Get file size in tar directory

def get_file_sizes(directory):
    file_sizes = {}
    
    # Loop over all files in a directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            
            # Get file size and add to dictionary
            size = os.path.getsize(file_path)
            file_sizes[file_path] = size
    
    return file_sizes

# Specify directory path
directory_path = save_dir

# Get file size
file_sizes = get_file_sizes(directory_path)

# Output file size
for file_path, size in file_sizes.items():
    print(f"{file_path}: {size} bytes")

/workspace/brail_poc3/experiments/Lemon_Raw/sub-032319.tar.gz: 241263339 bytes
/workspace/brail_poc3/experiments/Lemon_Raw/sub-032436.tar.gz: 257523176 bytes
/workspace/brail_poc3/experiments/Lemon_Raw/sub-032401.tar.gz: 251045147 bytes
/workspace/brail_poc3/experiments/Lemon_Raw/sub-032322.tar.gz: 243003957 bytes
/workspace/brail_poc3/experiments/Lemon_Raw/sub-032317.tar.gz: 243170567 bytes
/workspace/brail_poc3/experiments/Lemon_Raw/sub-032338.tar.gz: 238134994 bytes
/workspace/brail_poc3/experiments/Lemon_Raw/sub-032372.tar.gz: 251949206 bytes
/workspace/brail_poc3/experiments/Lemon_Raw/sub-032309.tar.gz: 93594799 bytes
/workspace/brail_poc3/experiments/Lemon_Raw/sub-032408.tar.gz: 264965712 bytes
/workspace/brail_poc3/experiments/Lemon_Raw/sub-032301.tar.gz: 243610492 bytes
/workspace/brail_poc3/experiments/Lemon_Raw/sub-032494.tar.gz: 241968057 bytes
/workspace/brail_poc3/experiments/Lemon_Raw/sub-032383.tar.gz: 245667729 bytes
/workspace/brail_poc3/experiments/Lemon_Raw/sub-03235

Step2: Fix file path

In [4]:
# File read function
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

# file write function
def write_file(file_path, content):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

# file path modification function
def fix_file_paths(vhdr_content, vmrk_content, prefix):
    # Get the actual file names of DataFile and MarkerFile
    data_file_match = re.search(r'DataFile=(\S+)', vhdr_content)
    marker_file_match = re.search(r'MarkerFile=(\S+)', vhdr_content)

    if data_file_match and marker_file_match:
        data_file_name = data_file_match.group(1)
        marker_file_name = marker_file_match.group(1)

        # log output
        print(f'Original DataFile: {data_file_name}')
        print(f'Original MarkerFile: {marker_file_name}')

        # Extract sub-032344 from file name
        sub_prefix = prefix

        # Fix DataFile
        vhdr_content = re.sub(r'DataFile=\S+', f'DataFile={sub_prefix}.eeg', vhdr_content)
        vmrk_content = re.sub(r'DataFile=\S+', f'DataFile={sub_prefix}.eeg', vmrk_content)

        # Fix MarkerFile
        vhdr_content = re.sub(r'MarkerFile=\S+', f'MarkerFile={sub_prefix}.vmrk', vhdr_content)

        # log output
        print(f'Fixed DataFile: {sub_prefix}.eeg')
        print(f'Fixed MarkerFile: {sub_prefix}.vmrk')

    return vhdr_content, vmrk_content

def paths(vhdr_path, vmrk_path):
#     vhdr_path = '/workspace/brail_poc3/experiments/Lemon_PreRaw/sub-032301/RSEEG/sub-032301.vhdr'
#     vmrk_path = '/workspace/brail_poc3/experiments/Lemon_PreRaw/sub-032301/RSEEG/sub-032301.vmrk'
    prefix = re.search(r'sub-\d+', vhdr_path).group(0)

    vhdr_content = read_file(vhdr_path)
    vmrk_content = read_file(vmrk_path)

    # Corrected file paths
    fixed_vhdr_content, fixed_vmrk_content = fix_file_paths(vhdr_content, vmrk_content, prefix)

    # Write the modifications to a file
    write_file(vhdr_path, fixed_vhdr_content)
    write_file(vmrk_path, fixed_vmrk_content)

In [5]:
vhdr_path_list = glob.glob(output_folder+'/*/RSEEG/*.vhdr')
vmrk_path_list = glob.glob(output_folder+'/*/RSEEG/*.vmrk')

for vhdr_path, vmrk_path in zip(vhdr_path_list, vmrk_path_list):
    paths(vhdr_path, vmrk_path)

Original DataFile: sub-010089.eeg
Original MarkerFile: sub-010089.vmrk
Fixed DataFile: sub-032376.eeg
Fixed MarkerFile: sub-032376.vmrk
Original DataFile: sub-010269.eeg
Original MarkerFile: sub-010269.vmrk
Fixed DataFile: sub-032476.eeg
Fixed MarkerFile: sub-032476.vmrk
Original DataFile: sub-010027.eeg
Original MarkerFile: sub-010027.vmrk
Fixed DataFile: sub-032319.eeg
Fixed MarkerFile: sub-032319.vmrk
Original DataFile: sub-010077.eeg
Original MarkerFile: sub-010077.vmrk
Fixed DataFile: sub-032365.eeg
Fixed MarkerFile: sub-032365.vmrk
Original DataFile: sub-010226.eeg
Original MarkerFile: sub-010226.vmrk
Fixed DataFile: sub-032434.eeg
Fixed MarkerFile: sub-032434.vmrk
Original DataFile: sub-010234.eeg
Original MarkerFile: sub-010234.vmrk
Fixed DataFile: sub-032442.eeg
Fixed MarkerFile: sub-032442.vmrk
Original DataFile: sub-010004.eeg
Original MarkerFile: sub-010004.vmrk
Fixed DataFile: sub-032303.eeg
Fixed MarkerFile: sub-032303.vmrk
Original DataFile: sub-010263.eeg
Original Marke

Original DataFile: sub-010279.eeg
Original MarkerFile: sub-010279.vmrk
Fixed DataFile: sub-032486.eeg
Fixed MarkerFile: sub-032486.vmrk
Original DataFile: sub-010152.eeg
Original MarkerFile: sub-010152.vmrk
Fixed DataFile: sub-032395.eeg
Fixed MarkerFile: sub-032395.vmrk
Original DataFile: sub-010064.eeg
Original MarkerFile: sub-010064.vmrk
Fixed DataFile: sub-032352.eeg
Fixed MarkerFile: sub-032352.vmrk
Original DataFile: sub-010305.eeg
Original MarkerFile: sub-010305.vmrk
Fixed DataFile: sub-032512.eeg
Fixed MarkerFile: sub-032512.vmrk
Original DataFile: sub-010207.eeg
Original MarkerFile: sub-010207.vmrk
Fixed DataFile: sub-032421.eeg
Fixed MarkerFile: sub-032421.vmrk
Original DataFile: sub-010200.eeg
Original MarkerFile: sub-010200.vmrk
Fixed DataFile: sub-032416.eeg
Fixed MarkerFile: sub-032416.vmrk
Original DataFile: sub-010287.eeg
Original MarkerFile: sub-010287.vmrk
Fixed DataFile: sub-032494.eeg
Fixed MarkerFile: sub-032494.vmrk
Original DataFile: sub-010245.eeg
Original Marke

Step3: EEG reading, resampling, filtering, croping

In [9]:
raw_list = glob.glob(output_folder+'/*/RSEEG/*.vhdr')
prepSignal_EC = mne.io.read_raw_eeglab(locFile_dir, preload=True)
passList = []

for raw_path in tqdm(raw_list):
    
    Sub = {}
    Sub['SubNum'] = raw_path.split('/')[-1].split('.')[0]
    if os.path.exists(f"{preprocessed_dir}/{Sub['SubNum']}.npy"):
        print(f"File exist, skip {Sub['SubNum']}")
        continue
    
    # Reading
    raw = mne.io.read_raw_brainvision(raw_path, preload=True)
    raw_fit = raw.copy()

    # Data resampling to 250 Hz
    raw_fit.resample(250)
    
    # Apply high-pass and low-pass filters
    raw_fit.filter(l_freq=1, h_freq=45, picks='eeg', method='fir', fir_design='firwin')
    # Plot and confirm
#     raw_fit.plot(n_channels=len(raw.ch_names), duration=10);

    # Copy ch loc info
    for raw_ch in raw_fit.info['chs']:
        # Search prepSignal_EC.info for the corresponding channel in 'ch_name
        prep_ch = next((ch for ch in prepSignal_EC.info['chs'] if ch['ch_name'] == raw_ch['ch_name']), None)
        # Copy to raw.info['chs'] if 'loc' attribute exists
        if prep_ch and 'loc' in prep_ch:
            raw_ch['loc'] = prep_ch['loc']
    
    # Annotation crop
    anno = raw_fit.annotations.to_data_frame()
    # Creation of data frames
    df = pd.DataFrame(anno)
    if df.shape[0] < 10:
        passList.append(Sub['SubNum'])
        continue
    # "New Segment/" is converted relative to the starting point of 0 seconds.
    df['onset'] = pd.to_datetime(df['onset'])
    df['relative_onset'] = (df['onset'] - df.loc[df['description'] == 'New Segment/', 'onset'].iloc[0]).dt.total_seconds()
    # Leave only the first consecutive "Stimulus/S210" or "Stimulus/S200"
    df['210start'] = ~(df['description'].eq(df['description'].shift())) & (df['description'].str.startswith('Stimulus/S210') | df['description'].str.startswith('Stimulus/S208'))
    df['210end'] = ~(df['description'].eq(df['description'].shift(-1))) & (df['description'].str.startswith('Stimulus/S210') | df['description'].str.startswith('Stimulus/S208'))
    df['200start'] = ~(df['description'].eq(df['description'].shift())) & df['description'].str.startswith('Stimulus/S200')
    df['200end'] = ~(df['description'].eq(df['description'].shift(-1))) & df['description'].str.startswith('Stimulus/S200')
    df210start_filtered = df[df['210start']].drop(columns='210start')
    df210end_filtered = df[df['210end']].drop(columns='210end')
    df200start_filtered = df[df['200start']].drop(columns='200start')
    df200end_filtered = df[df['200end']].drop(columns='200end')
    # Conversion to Dictionary
    start_time_EC = df210start_filtered['relative_onset'].to_list()
    end_time_EC = df210end_filtered['relative_onset'].to_list()
    start_time_EO = df200start_filtered['relative_onset'].to_list()
    end_time_EO = df200end_filtered['relative_onset'].to_list()
    # Display Results
    print(start_time_EO)
    print(end_time_EO)
    print(start_time_EC)
    print(end_time_EC)
    
    # Crop
    folder_path = f"{preprocessed_dir}/{Sub['SubNum']}/"
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    for num in range(len(start_time_EO)):
        raw_cropped_EO = raw_fit.copy().crop(tmin=start_time_EO[num], tmax=end_time_EO[num])
        raw_cropped_EC = raw_fit.copy().crop(tmin=start_time_EC[num], tmax=end_time_EC[num])
        raw_cropped_EO.save(f"{preprocessed_dir}/{Sub['SubNum']}/{Sub['SubNum']}_EO{num+1}_eeg.fif")
        raw_cropped_EO.save(f"{preprocessed_dir}/{Sub['SubNum']}/{Sub['SubNum']}_EC{num+1}_eeg.fif")
        
    # Save
#     np.save(f"/workspace/brail_poc3/experiments/Lemon_PreDict/{Sub['SubNum']}.npy", Sub)

Reading /workspace/brail_poc3/experiments/Lemon_PreEEGLAB/sub-032301/sub-032301/sub-032301_EC.fdt
Reading 0 ... 119344  =      0.000 ...   477.376 secs...


/tmp/ipykernel_2200047/64884789.py:2: RuntimeWarning: Data file name in EEG.data (sub-010002_EC.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-032301_EC.fdt).
  prepSignal_EC = mne.io.read_raw_eeglab('/workspace/brail_poc3/experiments/Lemon_PreEEGLAB/sub-032301/sub-032301/sub-032301_EC.set', preload=True)
/tmp/ipykernel_2200047/64884789.py:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  prepSignal_EC = mne.io.read_raw_eeglab('/workspace/brail_poc3/experiments/Lemon_PreEEGLAB/sub-032301/sub-032301/sub-032301_EC.set', preload=True)
/tmp/ipykernel_2200047/64884789.py:2: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  prepSignal_EC = mne.io.read_raw_eeglab('/workspace/brail_poc3/experiments/Lemon_PreEEGLAB/sub-032301/sub-032301/sub-032301_EC.set', preload=True)
  0%|                                         

Extracting parameters from /workspace/brail_poc3/experiments/Lemon_PreRaw/sub-032376/RSEEG/sub-032376.vhdr...
Setting channel info structure...
Reading 0 ... 2562899  =      0.000 ...  1025.160 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


[79.63640000000001, 204.7792, 329.97720000000004, 456.48400000000004, 580.4616000000001, 704.7896000000001, 831.9332, 957.0088000000001]
[137.63760000000002, 262.78040000000004, 387.978, 514.4848000000001, 638.4628, 762.7904000000001, 889.9340000000001, 1015.0096000000001]
[16.9904, 142.75240000000002, 268.1952, 392.086, 518.6828, 642.7696000000001, 767.8844, 894.21]
[74.9916, 200.75320000000002, 326.196, 450.08680000000004, 576.6836000000001, 700.7704, 825.8852, 952.2108000000001]
Writing /workspace/brail_poc3/experiments/Lemon_PreDict/sub-032376/sub-032376_EO1_eeg.fif
Closing /workspace/brail_poc3/experiments/Lemon_PreDict/sub-032376/sub-032376_EO1_eeg.fif
[done]
Writing /workspace/brail_poc3/experiments/Lemon_PreDict/sub-032376/sub-032376_EC1_eeg.fif
Closing /workspace/brail_poc3/experiments/Lemon_PreDict/sub-032376/sub-032376_EC1_eeg.fif
[done]


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


[67.80040000000001, 191.3232, 315.7516, 439.4384, 563.6916, 687.8364, 811.5968, 935.4920000000001]
[125.802, 249.32440000000003, 373.75320000000005, 497.43960000000004, 621.6932, 745.8376000000001, 869.5980000000001, 993.4932000000001]
[5.6372, 129.698, 254.2064, 377.78880000000004, 501.82360000000006, 625.6080000000001, 749.7356000000001, 873.5148]
[63.6388, 187.69920000000002, 312.2076, 435.79040000000003, 559.8248, 683.6092, 807.7368, 931.5164000000001]
Writing /workspace/brail_poc3/experiments/Lemon_PreDict/sub-032324/sub-032324_EO1_eeg.fif
Closing /workspace/brail_poc3/experiments/Lemon_PreDict/sub-032324/sub-032324_EO1_eeg.fif
[done]
Writing /workspace/brail_poc3/experiments/Lemon_PreDict/sub-032324/sub-032324_EC1_eeg.fif
Closing /workspace/brail_poc3/experiments/Lemon_PreDict/sub-032324/sub-032324_EC1_eeg.fif
[done]


[Parallel(n_jobs=1)]: Done  62 out of  62 | elapsed:    0.5s finished


Writing /workspace/brail_poc3/experiments/Lemon_PreDict/sub-032324/sub-032324_EO2_eeg.fif
Closing /workspace/brail_poc3/experiments/Lemon_PreDict/sub-032324/sub-032324_EO2_eeg.fif
[done]
Writing /workspace/brail_poc3/experiments/Lemon_PreDict/sub-032324/sub-032324_EC2_eeg.fif
Closing /workspace/brail_poc3/experiments/Lemon_PreDict/sub-032324/sub-032324_EC2_eeg.fif
[done]
Writing /workspace/brail_poc3/experiments/Lemon_PreDict/sub-032324/sub-032324_EO3_eeg.fif
Closing /workspace/brail_poc3/experiments/Lemon_PreDict/sub-032324/sub-032324_EO3_eeg.fif
[done]
Writing /workspace/brail_poc3/experiments/Lemon_PreDict/sub-032324/sub-032324_EC3_eeg.fif
Closing /workspace/brail_poc3/experiments/Lemon_PreDict/sub-032324/sub-032324_EC3_eeg.fif
[done]
Writing /workspace/brail_poc3/experiments/Lemon_PreDict/sub-032324/sub-032324_EO4_eeg.fif
Closing /workspace/brail_poc3/experiments/Lemon_PreDict/sub-032324/sub-032324_EO4_eeg.fif
[done]
Writing /workspace/brail_poc3/experiments/Lemon_PreDict/sub-03232

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 214/214 [49:50<00:00, 13.98s/it]


In [92]:
'Stimulus/S  1' in anno['description'].tolist()

True

In [95]:
passList

['sub-032476',
 'sub-032470',
 'sub-032399',
 'sub-032492',
 'sub-032478',
 'sub-032389']